In [19]:
import cv2
import numpy as np

def analyze_players(image_path):
    """
    Analyzes the players in an NBA game image to extract information
    relevant to foul detection, including body touch.
    """
    try:
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error: Could not open or find the image at {image_path}")
            return

        # 1. Object Detection (Conceptual)
        #    - Detect players. Output: Bounding boxes and player IDs.
        # detected_players = detect_players(img)
        detected_players = [
            {'id': 67, 'box': [100, 50, 150, 200]},
            {'id': 3, 'box': [200, 100, 250, 250]},
            {'id': 15, 'box': [300, 150, 350, 300]},
        ]

        # 2. Pose Estimation (Conceptual)
        #    - Estimate player poses. Output: Keypoints.
        # player_poses = estimate_player_poses(detected_players, img)
        player_poses = {
            67: {'keypoints': [[110, 70], [120, 100], [130, 130], [140, 160], [125, 180]]},
            3:  {'keypoints': [[210, 120], [220, 150], [230, 180], [240, 210], [225, 230]]},
            15: {'keypoints': [[310, 170], [320, 200], [330, 230], [340, 260], [325, 280]]},
        }

        # 3. Analyze Player Information
        player_info = {}
        for player in detected_players:
            player_id = player['id']
            player_box = player['box']
            player_pose = player_poses.get(player_id, None)

            x1, y1, x2, y2 = player_box
            width = x2 - x1
            height = y2 - y1
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2

            player_data = {
                'id': player_id,
                'box': player_box,
                'center': (center_x, center_y),
                'width': width,
                'height': height,
            }

            if player_pose:
                player_data['pose'] = player_pose['keypoints']

            player_info[player_id] = player_data

        # 4. Detect Body Touch Foul
        foul_detected = False
        if 67 in player_info and 3 in player_info:
            player_67_data = player_info[67]
            player_3_data = player_info[3]

            if 'pose' in player_67_data and 'pose' in player_3_data:
                proximity_threshold = 50  # Adjust as needed
                foul_detected = check_body_touch(player_67_data['pose'], player_3_data['pose'], proximity_threshold)

        # 5. Print Results
        for player_id, data in player_info.items():
            print(f"Player {player_id}:")
            print(f"  Bounding Box: {data['box']}")
            print(f"  Center: {data['center']}")
            print(f"  Width: {data['width']}, Height: {data['height']}")
            if 'pose' in data:
                print(f"  Pose Keypoints: {data['pose']}")
            else:
                print("  Pose information not available.")
            print("-" * 20)

        if foul_detected:
            print("Body touch foul detected between Player 67 and Player 3.")
        else:
            print("No body touch foul detected between Player 67 and Player 3.")

        return player_info

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def check_body_touch(pose1_keypoints, pose2_keypoints, threshold):
    """
    Checks if there is a body touch between two players based on their keypoints.
    """
    for i, point1 in enumerate(pose1_keypoints):
        for j, point2 in enumerate(pose2_keypoints):
            dist = np.linalg.norm(np.array(point1) - np.array(point2))
            if dist < threshold:
                return True  # Contact detected
    return False  # No contact detected
# Example usage
image_file = "C:/Users/YAMI/Downloads/basketball 3.jpeg"
player_data = analyze_players(image_file)



Player 67:
  Bounding Box: [100, 50, 150, 200]
  Center: (125, 125)
  Width: 50, Height: 150
  Pose Keypoints: [[110, 70], [120, 100], [130, 130], [140, 160], [125, 180]]
--------------------
Player 3:
  Bounding Box: [200, 100, 250, 250]
  Center: (225, 175)
  Width: 50, Height: 150
  Pose Keypoints: [[210, 120], [220, 150], [230, 180], [240, 210], [225, 230]]
--------------------
Player 15:
  Bounding Box: [300, 150, 350, 300]
  Center: (325, 225)
  Width: 50, Height: 150
  Pose Keypoints: [[310, 170], [320, 200], [330, 230], [340, 260], [325, 280]]
--------------------
No body touch foul detected between Player 67 and Player 3.


In [27]:
import cv2
import numpy as np
from sklearn.linear_model import LogisticRegression  # Example model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

def extract_features(image_path):
    """
    Extracts relevant features from an NBA game image for foul prediction.
    """
    try:
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error: Could not open or find the image at {image_path}")
            return None

        # 1. Object Detection (Conceptual)
        #    - Detect players and the ball.
        # detected_objects = detect_objects(img)
        detected_objects = [
            {'label': 'player', 'id': 67, 'box': [100, 50, 150, 200]},
            {'label': 'player', 'id': 3, 'box': [200, 100, 250, 250]},
            {'label': 'ball', 'box': [270, 80, 290, 100]},
        ]

        # 2. Pose Estimation (Conceptual)
        # player_poses = estimate_player_poses(detected_objects, img)
        player_poses = {
            67: {'keypoints': [[110, 70], [120, 100], [130, 130], [140, 160], [125, 180]]},
            3:  {'keypoints': [[210, 120], [220, 150], [230, 180], [240, 210], [225, 230]]},
        }

        # 3. Feature Extraction
        features = []
        player_features = {}
        for obj in detected_objects:
            if obj['label'] == 'player':
                player_id = obj['id']
                box = obj['box']
                center_x, center_y = (box[0] + box[2]) // 2, (box[1] + box[3]) // 2
                width = box[2] - box[0]
                height = box[3] - box[1]

                player_features[player_id] = {
                    'center_x': center_x,
                    'center_y': center_y,
                    'width': width,
                    'height': height,
                }
                if player_id in player_poses:
                    keypoints = player_poses[player_id]['keypoints']
                    for i, point in enumerate(keypoints):
                        player_features[player_id][f'keypoint_{i}_x'] = point[0]
                        player_features[player_id][f'keypoint_{i}_y'] = point[1]
        # Get features for player 67 and 3.
        if 67 in player_features and 3 in player_features:
            features.extend(list(player_features[67].values()))
            features.extend(list(player_features[3].values()))
        else:
            return None
        return features

    except Exception as e:
        print(f"An error occurred: {e}")
        return None



def create_model(data, labels):
    """
    Creates and trains a machine learning model for foul prediction.
    """
    # 1. Data Splitting
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

    # 2. Model Selection and Training
    model = LogisticRegression(solver='liblinear')  # Example: Logistic Regression
    model.fit(X_train, y_train)

    # 3. Model Evaluation
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model Accuracy: {accuracy:.2f}")
    print(classification_report(y_test, y_pred))

    return model

def predict_foul(image_path, model):
    """
    Predicts whether a foul will occur in a given image using the trained model.
    """
    features = extract_features(image_path)
    if features is None:
        return "Error: Could not extract features from the image."

    # Reshape the features to be a 2D array (1 sample, n features)
    features = np.array(features).reshape(1, -1)
    prediction = model.predict(features)
    probability = model.predict_proba(features) #get the probability of the prediction

    if prediction[0] == 1:
        return f"Foul Predicted! (Probability: {probability[0][1]:.2f})"
    else:
        return f"No Foul Predicted. (Probability: {probability[0][0]:.2f})"

def main():
    """
    Main function to train the model and predict fouls in new images.
    """
    # 1. Prepare Training Data (Crucially Important)
    #    - This is where you'd load your labeled dataset of images and labels.
    #    - Each image should be processed by extract_features() to get the feature vector.
    #    - Labels should be 0 (no foul) or 1 (foul).
    #    -  Example (conceptual):
    # training_data = [extract_features("train_image1.jpg"), extract_features("train_image2.jpg"), ...]
    # training_labels = [0, 1, 0, 1, ...]  # 0: no foul, 1: foul
    training_data = []
    training_labels = []
    # 2. Load the training data.
    for i in range(100): #generate dummy data
        image_file = "C:/Users/YAMI/Downloads/basketball 3.jpeg"
        features = extract_features(image_file)
        if features:
            training_data.append(features)
            training_labels.append(i%2) #alternate between 0 and 1
    if not training_data:
        print("Error: No training data available.  Cannot train model.")
        return

    # Convert to numpy arrays
    training_data = np.array(training_data)
    training_labels = np.array(training_labels)
    # 3. Create and Train the Model
    model = create_model(training_data, training_labels)

    # 4. Predict on a New Image
    new_image_path = "C:/Users/YAMI/Downloads/basketball 2.jpeg"  # Replace with a new image to test
    prediction = predict_foul(new_image_path, model)
    print(f"Prediction for {new_image_path}: {prediction}")


if __name__ == "__main__":
    main()


Model Accuracy: 0.40
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        12
           1       0.40      1.00      0.57         8

    accuracy                           0.40        20
   macro avg       0.20      0.50      0.29        20
weighted avg       0.16      0.40      0.23        20

Prediction for C:/Users/YAMI/Downloads/basketball 2.jpeg: Foul Predicted! (Probability: 0.52)


C:\Users\YAMI\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\YAMI\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\YAMI\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [29]:
import cv2
import numpy as np
from sklearn.linear_model import LogisticRegression  # Example model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

def extract_features(image_path):
    """
    Extracts relevant features from an NBA game image for foul prediction.
    """
    try:
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error: Could not open or find the image at {image_path}")
            return None

        # 1. Object Detection (Conceptual)
        #    - Detect players and the ball.
        # detected_objects = detect_objects(img)
        detected_objects = [
            {'label': 'player', 'id': 67, 'box': [100, 50, 150, 200]},
            {'label': 'player', 'id': 3, 'box': [200, 100, 250, 250]},
            {'label': 'ball', 'box': [270, 80, 290, 100]},
        ]

        # 2. Pose Estimation (Conceptual)
        # player_poses = estimate_player_poses(detected_objects, img)
        player_poses = {
            67: {'keypoints': [[110, 70], [120, 100], [130, 130], [140, 160], [125, 180]]},
            3:  {'keypoints': [[210, 120], [220, 150], [230, 180], [240, 210], [225, 230]]},
        }

        # 3. Feature Extraction
        features = []
        player_features = {}
        for obj in detected_objects:
            if obj['label'] == 'player':
                player_id = obj['id']
                box = obj['box']
                center_x, center_y = (box[0] + box[2]) // 2, (box[1] + box[3]) // 2
                width = box[2] - box[0]
                height = box[3] - box[1]

                player_features[player_id] = {
                    'center_x': center_x,
                    'center_y': center_y,
                    'width': width,
                    'height': height,
                }
                if player_id in player_poses:
                    keypoints = player_poses[player_id]['keypoints']
                    for i, point in enumerate(keypoints):
                        player_features[player_id][f'keypoint_{i}_x'] = point[0]
                        player_features[player_id][f'keypoint_{i}_y'] = point[1]

        # Get features for player 67 and 3.  Crucially changed.
        if 67 in player_features and 3 in player_features:
            player67_features = player_features[67]
            player3_features = player_features[3]

            # Calculate distance between centers
            distance = np.sqrt(
                (player67_features['center_x'] - player3_features['center_x']) ** 2 +
                (player67_features['center_y'] - player3_features['center_y']) ** 2
            )
            features.append(distance)  # Add the distance as a feature

            # Add other features, including keypoint distances
            features.extend(list(player67_features.values()))
            features.extend(list(player3_features.values()))

            #Add keypoint distances.
            if 'pose' in player_poses[67] and 'pose' in player_poses[3]:
                for i, point67 in enumerate(player_poses[67]['keypoints']):
                    for j, point3 in enumerate(player_poses[3]['keypoints']):
                         dist_kp = np.sqrt((point67[0]-point3[0])**2 + (point67[1]-point3[1])**2)
                         features.append(dist_kp) #add the distance between keypoints as a feature



        else:
            return None
        return features

    except Exception as e:
        print(f"An error occurred: {e}")
        return None



def create_model(data, labels):
    """
    Creates and trains a machine learning model for foul prediction.
    """
    # 1. Data Splitting
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

    # 2. Model Selection and Training
    model = LogisticRegression(solver='liblinear', class_weight='balanced')  #  Use balanced class weights
    model.fit(X_train, y_train)

    # 3. Model Evaluation
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model Accuracy: {accuracy:.2f}")
    print(classification_report(y_test, y_pred))

    return model

def predict_foul(image_path, model):
    """
    Predicts whether a foul will occur in a given image using the trained model.
    """
    features = extract_features(image_path)
    if features is None:
        return "Error: Could not extract features from the image."

    # Reshape the features
    features = np.array(features).reshape(1, -1)
    prediction = model.predict(features)
    probability = model.predict_proba(features) #get the probability of the prediction

    if prediction[0] == 1:
        return f"Foul Predicted! (Probability: {probability[0][1]:.2f})"
    else:
        return f"No Foul Predicted. (Probability: {probability[0][0]:.2f})"

def main():
    """
    Main function to train the model and predict fouls in new images.
    """
    # 1. Prepare Training Data
    training_data = []
    training_labels = []
    # Generate dummy data
    for i in range(100):
        image_file = "C:/Users/YAMI/Downloads/basketball 3.jpeg"
        features = extract_features(image_file)
        if features:
            training_data.append(features)
            training_labels.append(i % 2)

    if not training_data:
        print("Error: No training data available.  Cannot train model.")
        return

    # Convert to numpy arrays
    training_data = np.array(training_data)
    training_labels = np.array(training_labels)

    # 3. Create and Train the Model
    model = create_model(training_data, training_labels)

    # 4. Predict on a New Image
    new_image_path = "C:/Users/YAMI/Downloads/basketball 2.jpeg"
    prediction = predict_foul(new_image_path, model)
    print(f"Prediction for {new_image_path}: {prediction}")



if __name__ == "__main__":
    main()


Model Accuracy: 0.60
              precision    recall  f1-score   support

           0       0.60      1.00      0.75        12
           1       0.00      0.00      0.00         8

    accuracy                           0.60        20
   macro avg       0.30      0.50      0.38        20
weighted avg       0.36      0.60      0.45        20

Prediction for C:/Users/YAMI/Downloads/basketball 2.jpeg: No Foul Predicted. (Probability: 0.50)


C:\Users\YAMI\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\YAMI\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\YAMI\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [31]:
import cv2
import numpy as np
from sklearn.linear_model import LogisticRegression  # Example model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

def extract_features(image_path):
    """
    Extracts relevant features from an NBA game image for foul prediction.
    """
    try:
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error: Could not open or find the image at {image_path}")
            return None

        # 1. Object Detection (Conceptual)
        #    - Detect players and the ball.
        # detected_objects = detect_objects(img)
        detected_objects = [
            {'label': 'player', 'id': 67, 'box': [100, 50, 150, 200]},
            {'label': 'player', 'id': 3, 'box': [200, 100, 250, 250]},
            {'label': 'ball', 'box': [270, 80, 290, 100]},
        ]

        # 2. Pose Estimation (Conceptual)
        # player_poses = estimate_player_poses(detected_objects, img)
        player_poses = {
            67: {'keypoints': [[110, 70], [120, 100], [130, 130], [140, 160], [125, 180]]},
            3:  {'keypoints': [[210, 120], [220, 150], [230, 180], [240, 210], [225, 230]]},
        }

        # 3. Feature Extraction
        features = []
        player_features = {}
        for obj in detected_objects:
            if obj['label'] == 'player':
                player_id = obj['id']
                box = obj['box']
                center_x, center_y = (box[0] + box[2]) // 2, (box[1] + box[3]) // 2
                width = box[2] - box[0]
                height = box[3] - box[1]

                player_features[player_id] = {
                    'center_x': center_x,
                    'center_y': center_y,
                    'width': width,
                    'height': height,
                }
                if player_id in player_poses:
                    keypoints = player_poses[player_id]['keypoints']
                    for i, point in enumerate(keypoints):
                        player_features[player_id][f'keypoint_{i}_x'] = point[0]
                        player_features[player_id][f'keypoint_{i}_y'] = point[1]

        # Get features for player 67 and 3.
        if 67 in player_features and 3 in player_features:
            player67_features = player_features[67]
            player3_features = player_features[3]

            # Calculate distance between centers
            distance = np.sqrt(
                (player67_features['center_x'] - player3_features['center_x']) ** 2 +
                (player67_features['center_y'] - player3_features['center_y']) ** 2
            )
            features.append(distance)  # Add the distance as a feature

            # Add other features, including keypoint distances
            features.extend(list(player67_features.values()))
            features.extend(list(player3_features.values()))

            #Add keypoint distances.
            if 'pose' in player_poses[67] and 'pose' in player_poses[3]:
                for i, point67 in enumerate(player_poses[67]['keypoints']):
                    for j, point3 in enumerate(player_poses[3]['keypoints']):
                         dist_kp = np.sqrt((point67[0]-point3[0])**2 + (point67[1]-point3[1])**2)
                         features.append(dist_kp) #add the distance between keypoints as a feature



        else:
            return None
        return features

    except Exception as e:
        print(f"An error occurred: {e}")
        return None



def create_model(data, labels):
    """
    Creates and trains a machine learning model for foul prediction.
    """
    # 1. Data Splitting
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

    # 2. Model Selection and Training
    model = LogisticRegression(solver='liblinear', class_weight='balanced')  #  Use balanced class weights
    model.fit(X_train, y_train)

    # 3. Model Evaluation
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model Accuracy: {accuracy:.2f}")
    print(classification_report(y_test, y_pred))

    return model

def predict_foul(image_path, model):
    """
    Predicts whether a foul will occur in a given image using the trained model.
    """
    features = extract_features(image_path)
    if features is None:
        return "Error: Could not extract features from the image."

    # Reshape the features
    features = np.array(features).reshape(1, -1)
    prediction = model.predict(features)
    probability = model.predict_proba(features) #get the probability of the prediction

    if prediction[0] == 1:
        return f"Foul Predicted! (Probability: {probability[0][1]:.2f})"
    else:
        return f"No Foul Predicted. (Probability: {probability[0][0]:.2f})"

def main():
    """
    Main function to train the model and predict fouls in new images.
    """
    # 1. Prepare Training Data
    training_data = []
    training_labels = []
    # Generate dummy data
    for i in range(100):
        image_file = "C:/Users/YAMI/Downloads/basketball 3.jpeg"
        features = extract_features(image_file)
        if features:
            training_data.append(features)
            training_labels.append(i % 2)

    if not training_data:
        print("Error: No training data available.  Cannot train model.")
        return

    # Convert to numpy arrays
    training_data = np.array(training_data)
    training_labels = np.array(training_labels)

    # 3. Create and Train the Model
    model = create_model(training_data, training_labels)

    # 4. Predict on a New Image
    new_image_path = "C:/Users/YAMI/Downloads/basketball 2.jpeg"
    prediction = predict_foul(new_image_path, model)
    print(f"Prediction for {new_image_path}: {prediction}")

    # 5.  Print the prediction clearly
    if "Foul Predicted" in prediction:
        print("The model predicts a foul.")
    else:
        print("The model predicts no foul.")


if __name__ == "__main__":
    main()


Model Accuracy: 0.60
              precision    recall  f1-score   support

           0       0.60      1.00      0.75        12
           1       0.00      0.00      0.00         8

    accuracy                           0.60        20
   macro avg       0.30      0.50      0.38        20
weighted avg       0.36      0.60      0.45        20

Prediction for C:/Users/YAMI/Downloads/basketball 2.jpeg: No Foul Predicted. (Probability: 0.50)
The model predicts a foul.


C:\Users\YAMI\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\YAMI\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\YAMI\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
